In [64]:
import os
import pandas as pd
import numpy as np
import datetime as dt

In [65]:
ospwd = os.getcwd()

In [66]:
s = pd.read_pickle(ospwd + '\\deal_data20190508-2114\\s.pickle')
i0_aft_mov = pd.read_pickle(ospwd+'\\deal_data20190508-2114\\i0_aft_mov.pickle')
io_aft_mov = pd.read_pickle(ospwd+'\\deal_data20190508-2114\\io_aft_mov.pickle')
q_trans = pd.read_pickle(ospwd+'\\deal_data20190508-2114\\q_trans.pickle')
pi = pd.read_pickle(ospwd+'\\deal_data20190508-2114\\pi.pickle')
po = pd.read_pickle(ospwd+'\\deal_data20190508-2114\\po.pickle')

In [67]:
# i0_aft_mov.head(5)

In [68]:
# io_aft_mov.head(5)

In [69]:
# q_trans.head(5)

In [70]:
i0_aft_trans_area = i0_aft_mov.copy()
io_aft_trans_area = io_aft_mov.copy()

In [71]:
def skc_to_out(po,i0_aft_trans_area):
    i0_aft = i0_aft_trans_area.reset_index()

    i0_aft = pd.merge(i0_aft, io_aft_trans_area[['has_in']].reset_index(),on=['prod_id', 'color_id', 'size', 'org_id'], how='left')

    # i0_aft

    i0_aft = i0_aft[~i0_aft['org_id'].isin(['0001'])].copy() #删除仓库

    # i0_aft.head(5)

    pi1 = po.reset_index()[['prod_id', 'color_id', 'size','size_order']].drop_duplicates()

    i0_aft1 = i0_aft.copy()

    # i0_aft1.head(4)

    i0_aft1 = i0_aft1.loc[i0_aft1['i0_sum'] > 0 ]

    # i0_aft1.head(6)

    i01 = pd.merge(i0_aft1, pi1, how= 'inner')

    # i0_aft.head(7)

    i0_aft.loc[i0_aft['i0_sum'] > 0 , 'mark'] =1

    # i0_aft

    trans_skc = i0_aft[['prod_id', 'color_id','org_id', 'mark','i0_sum']].groupby(['prod_id', 'color_id', 'org_id']).sum().reset_index()

    # trans_skc = pd,merge()
    # trans_skc.loc[(trans_skc['prod_id']=='11219157334')& (trans_skc['color_id']=='002')&(trans_skc['org_id']=='8270')]

    trans_skc = trans_skc.loc[(trans_skc['mark']==1)&(trans_skc['i0_sum']==1)]

    trans_skc

    # trans_skc.to_csv(ospwd+ '//trans_skc.csv',encoding='gbk')

    trans_skc = trans_skc.reset_index()[['prod_id', 'color_id', 'org_id']].drop_duplicates()

    trans_skc

    trans_skc = pd.merge(trans_skc,
                             i0_aft[['prod_id', 'color_id', 'size', 'org_id', 'has_in', 'i0','i0_sum','r']],
                             on=['prod_id', 'color_id', 'org_id'], how='inner').drop_duplicates()

    # trans_skc.loc[trans_skc['i0_sum']>0]
    # trans_skc.head(5)
    return trans_skc

In [72]:
# trans_skc.to_csv(ospwd+ '//trans_skc11.csv',encoding='gbk')
# trans_skc1.to_csv(ospwd+ '//trans_skc22.csv',encoding='gbk')

In [73]:
#  trans_skc.loc[(trans_skc['i0_sum']>0) & (trans_skc['i0']==0)] #有在途，要换回去的
trans_skc = skc_to_out(po,i0_aft_mov)

In [74]:
 trans_skc.loc[(trans_skc['i0_sum']>0) & (trans_skc['i0']==0)] #有在途，要换回去的

,prod_id,color_id,org_id,size,has_in,i0,i0_sum,r
309,11219157334,002,8270,L,0.0,0.0,1.0,1.0
463,12119114072,001,7235,38,1.0,0.0,1.0,1.0
626,12219219093,202,X523,2XL,0.0,0.0,1.0,1.0
788,12219219406,204,X582,M,1.0,0.0,1.0,1.0
815,12219257490,003,X523,2XL,0.0,0.0,1.0,1.0


In [75]:
# trans_skc.head(5)
trans_skc.loc[(trans_skc['has_in']>0) ]

,prod_id,color_id,org_id,size,has_in,i0,i0_sum,r
463,12119114072,001,7235,38,1.0,0.0,1.0,1.0
788,12219219406,204,X582,M,1.0,0.0,1.0,1.0


### 返回库存

In [76]:
q = q_trans.copy()

In [77]:
# q.head(5)

In [78]:
s = s.reset_index('week_no')
s = s.loc[(s['week_no'] == -2) | (s['week_no'] == -1), ['s']].copy()
sales_skcs = s[['s']].reset_index(['prod_id', 'color_id', 'org_id']) \
    .groupby(['prod_id', 'color_id', 'org_id']).sum().reset_index()


In [79]:
# sales_skcs
# trans_skc

In [80]:
# i0_aft_trans_area['sum'] = i0_aft_trans_area['i0']+i0_aft_trans_area['r']
trans_skc_2 = trans_skc.loc[trans_skc['has_in'] == 1, :].set_index(['prod_id', 'color_id', 'size', 'org_id'])

In [81]:
trans_skc_2

,,,,has_in,i0,i0_sum,r
prod_id,color_id,size,org_id,,,,
12119114072,001,38,7235,1.0,0.0,1.0,1.0
12219219406,204,M,X582,1.0,0.0,1.0,1.0


In [82]:
# 'prod_id', 'color_id', 'size', 'org_send_id', 'org_rec_id', 'qty_mov'
q = q.set_index(['prod_id', 'color_id', 'size', 'org_rec_id',])
print('退还skus数和数量：', trans_skc.shape, q.loc[q.index.isin(trans_skc_2.index), 'qty_mov'].sum())

退还skus数和数量： (827, 8) 2.0


In [83]:
# q.loc[q.index.isin(trans_skc_2.index), 'qty_mov']

In [84]:
q = q[q['qty_mov']>0]

In [85]:
q = q.reset_index().rename(columns={'org_rec_id':'org_id'}).set_index(['prod_id', 'color_id', 'size', 'org_id'])

In [86]:
# q

### 更新

In [87]:
i0_aft_mov.reset_index().loc[(i0_aft_mov.reset_index()['prod_id']=='12119114072')&(i0_aft_mov.reset_index()['org_id']=='7235')]

,prod_id,color_id,size,org_id,i0,r,i0_sum
74752,12119114072,001,35,7235,0.0,0.0,0.0
74785,12119114072,001,36,7235,0.0,0.0,0.0
74818,12119114072,001,37,7235,0.0,0.0,0.0
74851,12119114072,001,38,7235,0.0,1.0,1.0
74884,12119114072,001,39,7235,0.0,0.0,0.0
74920,12119114072,204,35,7235,0.0,0.0,0.0
74965,12119114072,204,36,7235,1.0,0.0,1.0
75010,12119114072,204,37,7235,0.0,1.0,1.0
75055,12119114072,204,38,7235,6.0,0.0,6.0
75100,12119114072,204,39,7235,2.0,0.0,2.0


In [88]:
i0_aft_mov.reset_index().loc[(i0_aft_mov.reset_index()['prod_id']=='12119114072')&(i0_aft_mov.reset_index()['org_id']=='X011')]

,prod_id,color_id,size,org_id,i0,r,i0_sum
74763,12119114072,001,35,X011,0.0,0.0,0.0
74796,12119114072,001,36,X011,0.0,0.0,0.0
74829,12119114072,001,37,X011,0.0,0.0,0.0
74862,12119114072,001,38,X011,2.0,0.0,2.0
74895,12119114072,001,39,X011,1.0,0.0,1.0
74939,12119114072,204,35,X011,0.0,0.0,0.0
74984,12119114072,204,36,X011,1.0,0.0,1.0
75029,12119114072,204,37,X011,1.0,0.0,1.0
75074,12119114072,204,38,X011,2.0,0.0,2.0
75119,12119114072,204,39,X011,0.0,0.0,0.0


In [89]:
# i0_aft_mov

In [90]:
i0_aft_trans_area = i0_aft_mov.copy()

# if not trans_skc_2.empty:
q_rec_org = q.loc[~q.index.isin(trans_skc_2.index)].reset_index()
q_re = q.loc[q.index.isin(trans_skc_2.index)].reset_index()
q_re_send = q_re[['prod_id', 'color_id', 'size','org_send_id','qty_mov']].copy()
q_re_send = q_re_send.groupby(['prod_id', 'color_id', 'size', 'org_send_id']).sum()['qty_mov'].reset_index()
q_re_send = q_re_send.rename(columns={'org_send_id': 'org_id'})

In [91]:
# q_rec_org
q_re_send

,prod_id,color_id,size,org_id,qty_mov
0,12119114072,001,38,X011,1.0
1,12219219406,204,M,7325,1.0


In [92]:
#   更新调出库存
i0_aft_trans_area = pd.merge(i0_aft_trans_area,q_re_send,how='left',
                             on=['prod_id', 'color_id', 'size', 'org_id']).fillna(0)
i0_aft_trans_area['i0'] = i0_aft_trans_area['i0']+i0_aft_trans_area['qty_mov']
del i0_aft_trans_area['qty_mov']
# i0_aft_trans_area['sum'] = i0_aft_trans_area['i0']+i0_aft_trans_area['r']
i0_aft_trans_area['i0_sum'] = i0_aft_trans_area['i0']+i0_aft_trans_area['r']


In [93]:
#   更新调入库存
q_re_rec=q_re[['prod_id', 'color_id', 'size','org_id','qty_mov']].copy()
q_re_rec = q_re_rec.groupby(['prod_id', 'color_id', 'size', 'org_id']).sum()['qty_mov'].reset_index()
i0_aft_trans_area = pd.merge(i0_aft_trans_area,q_re_rec,
                             on=['prod_id', 'color_id', 'size', 'org_id'],how='left').fillna(0)
i0_aft_trans_area['r'] = i0_aft_trans_area['r']-i0_aft_trans_area['qty_mov']
del i0_aft_trans_area['qty_mov']
i0_aft_trans_area['i0_sum'] = i0_aft_trans_area['i0']+i0_aft_trans_area['r']
i0_aft_trans_area.set_index(['prod_id', 'color_id', 'size', 'org_id'], inplace=True)


In [94]:
# 更新调入调出
io_aft_trans_area= pd.merge(io_aft_trans_area.reset_index(),q_re_rec,how='left',
                            on=['prod_id', 'color_id', 'size', 'org_id']).fillna(0)
io_aft_trans_area.loc[io_aft_trans_area['qty_mov']>0,'has_in']=0
del io_aft_trans_area['qty_mov']
io_aft_trans_area.set_index(['prod_id','color_id','size','org_id'], inplace=True)

In [95]:
# io_aft_trans_area

In [96]:
q.loc[q.index.isin(trans_skc_2.index), 'qty_mov'] = 0

In [97]:
q = q[q.qty_mov > 0]

In [98]:
i0_aft = i0_aft_trans_area.copy()

In [99]:
trans_skc = skc_to_out(po,i0_aft)

In [100]:
# trans_skc

In [101]:
 trans_skc.loc[(trans_skc['has_in']>0)]

,prod_id,color_id,org_id,size,has_in,i0,i0_sum,r


In [102]:
i0_aft_trans_area.reset_index().loc[(i0_aft_trans_area.reset_index()['prod_id']=='12119114072')&(i0_aft_trans_area.reset_index()['org_id']=='7235')]

,prod_id,color_id,size,org_id,i0,r,i0_sum
74752,12119114072,001,35,7235,0.0,0.0,0.0
74785,12119114072,001,36,7235,0.0,0.0,0.0
74818,12119114072,001,37,7235,0.0,0.0,0.0
74851,12119114072,001,38,7235,0.0,0.0,0.0
74884,12119114072,001,39,7235,0.0,0.0,0.0
74920,12119114072,204,35,7235,0.0,0.0,0.0
74965,12119114072,204,36,7235,1.0,0.0,1.0
75010,12119114072,204,37,7235,0.0,1.0,1.0
75055,12119114072,204,38,7235,6.0,0.0,6.0
75100,12119114072,204,39,7235,2.0,0.0,2.0


In [103]:
i0_aft_trans_area.reset_index().loc[(i0_aft_trans_area.reset_index()['prod_id']=='12119114072')&(i0_aft_trans_area.reset_index()['org_id']=='X011')]

,prod_id,color_id,size,org_id,i0,r,i0_sum
74763,12119114072,001,35,X011,0.0,0.0,0.0
74796,12119114072,001,36,X011,0.0,0.0,0.0
74829,12119114072,001,37,X011,0.0,0.0,0.0
74862,12119114072,001,38,X011,3.0,0.0,3.0
74895,12119114072,001,39,X011,1.0,0.0,1.0
74939,12119114072,204,35,X011,0.0,0.0,0.0
74984,12119114072,204,36,X011,1.0,0.0,1.0
75029,12119114072,204,37,X011,1.0,0.0,1.0
75074,12119114072,204,38,X011,2.0,0.0,2.0
75119,12119114072,204,39,X011,0.0,0.0,0.0


### 已经将 库存返回

### 找到 发生调拨关系的门店

In [104]:
# q

In [105]:
q = q.reset_index().rename(columns={'org_id': 'org_rec_id'}) \
        .set_index(['prod_id', 'color_id', 'size', 'org_send_id', 'org_rec_id'])

In [106]:
q.head(4)

qty_mov  date_send
prod_id     color_id size org_send_id org_rec_id                    
11119101014 001      39   2822        7541            1.0 2019-05-06
                          3607        6099            1.0 2019-05-06
                          6095        X512            1.0 2019-05-06
                          6814        3771            1.0 2019-05-06

In [107]:
transq = q.reset_index(['org_rec_id', 'size'])

In [108]:
transq.head(6)

size org_rec_id  qty_mov  date_send
prod_id     color_id org_send_id                                    
11119101014 001      2822          39       7541      1.0 2019-05-06
                     3607          39       6099      1.0 2019-05-06
                     6095          39       X512      1.0 2019-05-06
                     6814          39       3771      1.0 2019-05-06
                     7233          39       7235      1.0 2019-05-06
                     7293          39       8792      1.0 2019-05-06

In [109]:
transq.rename_axis(['prod_id', 'color_id', 'org_id'], inplace=True)
transq.sort_index(inplace=True)

In [110]:
transq.head(5)

size org_rec_id  qty_mov  date_send
prod_id     color_id org_id                                    
11119101014 001      2822     39       7541      1.0 2019-05-06
                     2822     42       7541      1.0 2019-05-06
                     3131     40       X022      2.0 2019-05-06
                     3607     39       6099      1.0 2019-05-06
                     3607     43       9181      1.0 2019-05-06

In [111]:
# transq.head(4)

In [112]:
# i0_aft.head(6)

In [113]:
trans_skc.head(6)

,prod_id,color_id,org_id,size,has_in,i0,i0_sum,r
0,11119101014,004,7235,39,0.0,0.0,0.0,0.0
1,11119101014,004,7235,40,0.0,1.0,1.0,0.0
2,11119101014,004,7235,41,0.0,0.0,0.0,0.0
3,11119101014,004,7235,42,0.0,0.0,0.0,0.0
4,11119101014,004,7235,43,0.0,0.0,0.0,0.0
5,11119102162,001,X593,39,0.0,0.0,0.0,0.0


In [114]:
i0_aft.head(5)

i0    r  i0_sum
prod_id     color_id size org_id                  
11119101014 001      39   0001    0.0  0.0     0.0
                          2492    2.0  0.0     2.0
                          2822    4.0  0.0     4.0
                          3131    2.0  0.0     2.0
                          3607    1.0  0.0     1.0

### 有库存非仓库

In [115]:
# 有库存非仓库
i0_aft_copy = i0_aft.reset_index().copy()
i0_aft_copy.head(4)

,prod_id,color_id,size,org_id,i0,r,i0_sum
0,11119101014,001,39,0001,0.0,0.0,0.0
1,11119101014,001,39,2492,2.0,0.0,2.0
2,11119101014,001,39,2822,4.0,0.0,4.0
3,11119101014,001,39,3131,2.0,0.0,2.0


In [118]:
i0_aft_copy

,prod_id,color_id,size,org_id,i0,r,i0_sum
0,11119101014,001,39,0001,0.0,0.0,0.0
1,11119101014,001,39,2492,2.0,0.0,2.0
2,11119101014,001,39,2822,4.0,0.0,4.0
3,11119101014,001,39,3131,2.0,0.0,2.0
4,11119101014,001,39,3607,1.0,0.0,1.0
5,11119101014,001,39,3698,1.0,0.0,1.0
6,11119101014,001,39,3766,2.0,0.0,2.0
7,11119101014,001,39,3771,0.0,1.0,1.0
8,11119101014,001,39,5797,2.0,0.0,2.0
9,11119101014,001,39,6095,3.0,0.0,3.0


In [119]:
i0_aft_copy = pd.merge(i0_aft_copy, trans_skc[['prod_id', 'color_id']].drop_duplicates(), how='inner') #变大
# i0_aft_copy = i0_aft_copy.loc[~i0_aft_copy['org_id'].isin(wi.index)]

In [121]:
# i0_aft_copy

In [122]:
i0_aft_copy.head(5)

,prod_id,color_id,size,org_id,i0,r,i0_sum
0,11119101014,004,39,0001,0.0,0.0,0.0
1,11119101014,004,39,6095,0.0,0.0,0.0
2,11119101014,004,39,7233,7.0,0.0,7.0
3,11119101014,004,39,7235,0.0,0.0,0.0
4,11119101014,004,39,7312,2.0,0.0,2.0


In [123]:
i0_aft_copy = i0_aft_copy[~i0_aft_copy['org_id'].isin(['0001'])].copy() #删除仓库
i0_aft_copy.head(5)

,prod_id,color_id,size,org_id,i0,r,i0_sum
1,11119101014,004,39,6095,0.0,0.0,0.0
2,11119101014,004,39,7233,7.0,0.0,7.0
3,11119101014,004,39,7235,0.0,0.0,0.0
4,11119101014,004,39,7312,2.0,0.0,2.0
5,11119101014,004,39,7315,1.0,0.0,1.0


In [125]:
# i0_aft_copy

In [126]:
i0_aft_copy1 = i0_aft_copy.groupby(['prod_id', 'color_id', 'org_id']).sum()['i0_sum'].reset_index()

In [129]:
i0_aft_copy1

,prod_id,color_id,org_id,i0_sum
0,11119101014,004,6095,17.0
1,11119101014,004,7233,12.0
2,11119101014,004,7235,1.0
3,11119101014,004,7312,14.0
4,11119101014,004,7315,7.0
5,11119101014,004,7385,11.0
6,11119101014,004,7541,9.0
7,11119101014,004,8060,5.0
8,11119101014,004,9181,9.0
9,11119101014,004,X002,11.0


In [131]:
i0_aft_copy1 = i0_aft_copy1.loc[i0_aft_copy1['i0_sum'] > 0]
i0_aft_copy1.head(5)

,prod_id,color_id,org_id,i0_sum
0,11119101014,004,6095,17.0
1,11119101014,004,7233,12.0
2,11119101014,004,7235,1.0
3,11119101014,004,7312,14.0
4,11119101014,004,7315,7.0


In [132]:
i0_aft_copy = pd.merge(i0_aft_copy, i0_aft_copy1[['prod_id', 'color_id', 'org_id']], how='inner')

In [138]:
# i0_aft_copy

In [135]:
trans_skc['empty']=1

In [139]:
rec_may_store = pd.merge(i0_aft_copy, trans_skc[['prod_id', 'color_id', 'org_id', 'empty']],
                                 how='left',on=['prod_id', 'color_id', 'org_id']).fillna(0)

In [141]:
# rec_may_store

In [142]:
# 尺码未调出过
if not transq.empty:
    rec_may_store = pd.merge(rec_may_store,
                             transq.reset_index()[['prod_id', 'color_id', 'size', 'org_id', 'qty_mov']]
                             .drop_duplicates(subset=['prod_id', 'color_id', 'size', 'org_id']),
                             how='left',on=['prod_id', 'color_id', 'size', 'org_id']).reset_index().fillna(0)
    rec_may_store = rec_may_store.loc[rec_may_store['qty_mov']==0]

In [144]:
# rec_may_store

In [145]:
rec_may_store.head(4)

,index,prod_id,color_id,size,org_id,i0,r,i0_sum,empty,qty_mov
0,0,11119101014,004,39,6095,0.0,0.0,0.0,0.0,0.0
1,1,11119101014,004,40,6095,3.0,0.0,3.0,0.0,0.0
2,2,11119101014,004,41,6095,5.0,0.0,5.0,0.0,0.0
3,3,11119101014,004,42,6095,6.0,0.0,6.0,0.0,0.0


In [148]:
# sales_skcs

In [150]:
rec_may_store = pd.merge(rec_may_store[['prod_id', 'color_id', 'size', 'org_id','i0_sum']], sales_skcs, how='left').fillna(0)

In [152]:
rec_may_store = rec_may_store.loc[rec_may_store['s'] > 0]

In [159]:
rec_may_store.head(5)

,prod_id,color_id,size,org_id,i0_sum,s
0,11119101014,004,39,6095,0.0,1.0
1,11119101014,004,40,6095,3.0,1.0
2,11119101014,004,41,6095,5.0,1.0
3,11119101014,004,42,6095,6.0,1.0
4,11119101014,004,43,6095,3.0,1.0


In [157]:
rec_may_store1 = rec_may_store.groupby(['prod_id', 'color_id','org_id']).sum()['i0_sum'].reset_index()

In [160]:
rec_may_store1.head(5)

,prod_id,color_id,org_id,i0_sum
0,11119101014,004,6095,17.0
1,11119101014,004,7233,12.0
2,11119101014,004,7235,5.0
3,11119101014,004,7312,14.0
4,11119101014,004,7315,7.0
